# Combine Data Set

In [1]:
import torch
import pandas as pd
import pickle as pkl
from module.evaluate import load_dev_labels, get_metrics
import numpy as np

# import torch
# from model.highlight_ha import HierarchicalAttPredictorHL
# from config.basic_config import configs as config
# from torchmoji.global_variables import PRETRAINED_PATH, VOCAB_PATH
# import random
# from module.preprocessor import EnglishPreProcessor
# from pytorch_pretrained_bert.tokenization import BertTokenizer
# import json
# from torchmoji.sentence_tokenizer import SentenceTokenizer
# from torch.utils.data import Dataset, DataLoader
# from module import create_data

## read in raw test files to review

In [3]:
Wiki_test_file = pd.read_csv('/data/ToxicityDataOld/test_with_labels.csv')
IMDB_test_file = pd.read_csv('/data/SuperMod/imdb_test.csv')
Enron_test_file = pd.read_csv('/data/SuperMod/data_supermod_test.csv')


In [10]:
Wiki_test_file.columns = ['Unnamed: 0','comment_text' , 'toxic']

Index(['Unnamed: 0', 'id', 'comment_text', 'id.1', 'toxic', 'severe_toxic',
       'obscene', 'threat', 'insult', 'identity_hate'],
      dtype='object')

In [12]:
Wiki_test_file = Wiki_test_file[['Unnamed: 0','comment_text' , 'toxic']]
Wiki_test_file.columns =['Unnamed: 0','comment_text' , 'toxicity']
print("size wiki test", len(Wiki_test_file))
Wiki_test_file.head()

size wiki test 63978


,Unnamed: 0,comment_text,toxicity
0,5,Thank you for understanding. I think very high...,0
1,7,:Dear god this site is horrible.,0
2,11,"""::: Somebody will invariably try to add Relig...",0
3,13,""" \n\n It says it right there that it IS a typ...",0
4,14,""" \n\n == Before adding a new product to the l...",0


In [7]:
print("size wiki test", len(IMDB_test_file))
IMDB_test_file.head()

size wiki test 5000


,Unnamed: 0,comment_text,toxicity
0,40552,I gave this film my rare 10 stars.<br /><br />...,0
1,6106,This is one of the best films I have seen in y...,0
2,27831,"About halfway through, I realized I didn't car...",1
3,28253,The only good part of this movie was the endin...,1
4,37221,"After reading several comments, I felt I had t...",0


In [8]:
print("size wiki test", len(Enron_test_file))
Enron_test_file.head()

size wiki test 820


,Unnamed: 0,comment_text,toxicity
0,1455,MEN.....Stop Being Ashamed Of Your Penis Size!...,1
1,2971,all yoursteresa bosienatenron02/21/2001 04:07 ...,0
2,4255,This type of shit always happens to Crowell.\r...,1
3,5504,thanks doc sounds like fun see you saturday!,0
4,3867,a psych student could write a dissertation abo...,0


In [14]:
Enron_test_file_1 = Enron_test_file.sample(n=500, replace=True, random_state=1)

In [16]:
print("test len", len(Enron_test_file_1))
Enron_test_file_1.head()

test len 500


,Unnamed: 0,comment_text,toxicity
37,7938,barkerdeatnmenergy.com writes to the nyiso_tec...,0
235,3273,"mick/richard,can you advise me on the current ...",0
72,2223,"ken, attached are my credentials. i look forw...",0
767,1002,What are you talking about? You sent me some ...,1
715,7302,are you going to join us for lunch?sent from m...,0


## function to create one fold

In [23]:
def create_one_fold (size_each_data, seed):
    """create a test data with n = size_each_data 
    from each of the 3 test datasets"""
    w = Wiki_test_file.sample(n=500, replace=True, random_state=seed)
    e = Enron_test_file.sample(n=500, replace=True, random_state=seed)
    i = IMDB_test_file.sample(n=500, replace=True, random_state=seed)
    return pd.concat([w, e, i], axis=0)
    

In [24]:
test1 = create_one_fold(500,0)
print("test1 len", len(test1))
test1.head()

test1 len 1500


,Unnamed: 0,comment_text,toxicity
2732,6590,== Your (re)appointment == \n\n You have both ...,0
43567,104026,:Thank you for pointing those issues out. They...,0
42613,101810,== Trackers Task Force ==,0
52416,125175,"== isotope discrimination, atmospheric values,...",0
45891,109648,", because they're gay",0


In [26]:
test2 = create_one_fold(500,2)
print("test2 len", len(test2))
test2.head()

test2 len 1500


,Unnamed: 0,comment_text,toxicity
23720,56856,This is bloody ridiculous.,0
35343,84648,[[ Macmusmackie While rea...,0
6637,15969,This entire silly section should be removed. I...,0
30280,72522,== Your article == \n\n I've deleted it becaus...,0
44566,106382,== Thanks == \n\n I am still trying to work th...,0


## loop to create 50 folds

In [30]:
for i in range(50):
    # use i as seed
    outfile = create_one_fold(500,i)
    outfile_name = "/data/SuperMod/testfolds/fold" + str(i) +".csv"
    print(outfile_name)
    outfile.to_csv(outfile_name)

/data/SuperMod/testfolds/fold0.csv
/data/SuperMod/testfolds/fold1.csv
/data/SuperMod/testfolds/fold2.csv
/data/SuperMod/testfolds/fold3.csv
/data/SuperMod/testfolds/fold4.csv
/data/SuperMod/testfolds/fold5.csv
/data/SuperMod/testfolds/fold6.csv
/data/SuperMod/testfolds/fold7.csv
/data/SuperMod/testfolds/fold8.csv
/data/SuperMod/testfolds/fold9.csv
/data/SuperMod/testfolds/fold10.csv
/data/SuperMod/testfolds/fold11.csv
/data/SuperMod/testfolds/fold12.csv
/data/SuperMod/testfolds/fold13.csv
/data/SuperMod/testfolds/fold14.csv
/data/SuperMod/testfolds/fold15.csv
/data/SuperMod/testfolds/fold16.csv
/data/SuperMod/testfolds/fold17.csv
/data/SuperMod/testfolds/fold18.csv
/data/SuperMod/testfolds/fold19.csv
/data/SuperMod/testfolds/fold20.csv
/data/SuperMod/testfolds/fold21.csv
/data/SuperMod/testfolds/fold22.csv
/data/SuperMod/testfolds/fold23.csv
/data/SuperMod/testfolds/fold24.csv
/data/SuperMod/testfolds/fold25.csv
/data/SuperMod/testfolds/fold26.csv
/data/SuperMod/testfolds/fold27.csv
/d

In [33]:
load_test = pd.read_csv("/data/SuperMod/testfolds/fold49.csv")
print("len load_test", len(load_test))
print(load_test.iloc[1200])
load_test.head()

len load_test 1500
Unnamed: 0                                                   2713
Unnamed: 0.1                                                  586
comment_text    I really liked this movie. I have seen several...
toxicity                                                        0
Name: 1200, dtype: object


,Unnamed: 0,Unnamed: 0.1,comment_text,toxicity
0,426,991,""" \n ::::::Are you kidding: """"I looked in the ...",0
1,54957,131263,:The image seems to be find. We improved the c...,0
2,29096,69658,"==Thanks== \n Hello Patchy1, \n I want briefly...",0
3,15861,38033,==Study: Germany 1954 World Cup winners 'were ...,0
4,19638,47056,""" \n\n It only became a pejorative nickname fo...",0


In [34]:
load_test = pd.read_csv("/data/SuperMod/testfolds/fold20.csv")
print("len load_test", len(load_test))
print(load_test.iloc[1200])
load_test.head()

len load_test 1500
Unnamed: 0                                                   1216
Unnamed: 0.1                                                48626
comment_text    i two came home from school fast as i could to...
toxicity                                                        0
Name: 1200, dtype: object


,Unnamed: 0,Unnamed: 0.1,comment_text,toxicity
0,48483,115709,Hello nodutschke you better stop this war agai...,0
1,27098,64862,"""== VP70 in Popular Culture == \n **I'm pullin...",0
2,37135,88877,What the hell are you sending me vandalism war...,0
3,23775,56975,=== Listing note ===,0
4,23452,56170,== Shut up == \n\n See above.,1
